In [3]:
## IMPOR RELEVANT LIBRAIES

# DATA WRANGLING AND ANALYSIS
import pandas as pd
import missingno
from   skimpy import skim
import datetime as dt
import numpy as np

# DATA VISULIZATION 
import seaborn  as sns
import matplotlib.pyplot as plt

# DATE MANIPULATION
#from dateutil.relativedelta import relativedelta

In [4]:
# import relevant datasets 

# CUSTOMER DEMOGRAPHIC DATASET
cusDemo    = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 3 , parse_dates=True)

#Transation DATA
transc     = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 1, parse_dates=True) 

#CUSTOMER ADDRESS
cusAddress = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 4, parse_dates=True) 


C:\Users\Owner\AppData\Local\Temp\ipykernel_14572\505372198.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  cusDemo    = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 3 , parse_dates=True)


#### DATA CLEANING - CUSTOMER DEMOGRAPHICS

In [6]:
cusDemo_cleaned = cusDemo.copy() 
cusDemo_cleaned.drop(columns = ['first_name', 'last_name', 'default'], inplace = True)
y  = 2017    
cusDemo_cleaned.DOB.fillna(cusDemo_cleaned.DOB.mode()[0], inplace = True)
cusDemo_cleaned.DOB = round(y - cusDemo_cleaned.DOB.dt.year).astype(int)
cusDemo_cleaned = cusDemo_cleaned[cusDemo_cleaned.DOB < 100]

,customer_id,past_3_years_bike_related_purchases,DOB,tenure
count,3999.000000,3999.000000,3999.000000,3912.000000
mean,2000.991748,48.887472,39.914729,10.654652
std,1154.570372,28.718151,12.479958,5.658898
min,1.000000,0.000000,15.000000,1.000000
25%,1001.500000,24.000000,31.000000,6.000000
50%,2001.000000,48.000000,40.000000,11.000000
75%,3000.500000,73.000000,49.000000,15.000000
max,4000.000000,99.000000,86.000000,22.000000


In [7]:
cusDemo_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          3999 non-null   int64  
 1   gender                               3999 non-null   object 
 2   past_3_years_bike_related_purchases  3999 non-null   int64  
 3   DOB                                  3999 non-null   int32  
 4   job_title                            3493 non-null   object 
 5   job_industry_category                3343 non-null   object 
 6   wealth_segment                       3999 non-null   object 
 7   deceased_indicator                   3999 non-null   object 
 8   owns_car                             3999 non-null   object 
 9   tenure                               3912 non-null   float64
dtypes: float64(1), int32(1), int64(2), object(6)
memory usage: 328.0+ KB


In [9]:
bins    = np.arange(10,90,5)
labels_ = np.arange(10,90,5)
cusDemo_cleaned['AgeClass'] = pd.cut(cusDemo_cleaned.DOB, bins, labels = labels_)
cusDemo_cleaned.AgeClass.value_counts()

ValueError: Bin labels must be one fewer than the number of bin edges

In [ ]:
cusDemo_cleaned = cusDemo.copy() 

##### DROP IRRELEVANT COLUMNS
'''
Drop  default column, i doesnt have a meaning
Drop  default first and last names, Name will not influence purchase pattern
'''
cusDemo_cleaned.drop(columns = ['first_name', 'last_name', 'default'], inplace = True)
#### CLEAN DATE OF BIRTH COLUMN
""" 
- Convert Date of birth to Age 
- Drop null values
- Find and drop outliers
"""

# CONVERT DATE OF BIRTH TO AGE AND CLASSIFY IN TENS 
#take the maximum transcatodedate as the time of the project 
y  = 2017    
"""Fill missing age with mode"""
cusDemo_cleaned.DOB.fillna(cusDemo_cleaned.DOB.mode()[0], inplace = True)
cusDemo_cleaned.DOB = round(y - cusDemo_cleaned.DOB.dt.year).astype(int)
#It seen there is an outlier Age, Lets find out if there are more than one customer that is more than  100 years old 

"""
Oops!, her just caught an oulier.
In a real life situation, we can verify the customer age bu in this case, i will drop the customer from our database 
and the null date of births
"""
cusDemo_cleaned = cusDemo_cleaned[cusDemo_cleaned.DOB < 100]
"""Classify age into bins"""




# Clean job_title','job_industry_category Columns
""""
- Find cell is missing values
- Drop nulls in both category ---> 'job_title','job_industry_category'
"""


cusDemo_cleaned.job_industry_category.isna().value_counts()
cusDemo_cleaned.dropna(how = 'any')

# Attain unique job title to job industry 
job_to_title = cusDemo_cleaned[['job_title','job_industry_category']].dropna().drop_duplicates().reset_index(drop = True)

job_to_title[job_to_title.job_title == 'Blank']
              
test = cusDemo[['customer_id','DOB']].dropna()
test['AGE'] = 2017 - test.DOB.dt.year
test.AGE.astype(int)

test = test[test != test.AGE.max()]

bins = np.arange(10,100,10)
labels_ = np.arange(10,90,10)
test['Grouping'] = pd.cut(test.AGE, bins, labels = labels_ )
test


#### DATA CLEANING - address data 
#### DATA MERGING  - address and demographics data 

In [ ]:
cusAddress_cleaned = cusAddress.copy() 

#### DATA CLEANING - transcation data 

In [46]:
# Copy the transaaction data 
trans_cleaned = transc.copy()

skim(trans_cleaned)

#Did customerss buy similar items??  - different items were purchased at diffrent timers 
trans_cleaned[trans_cleaned.customer_id == 1]

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types                                           │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                    │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                    │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                    │
│ │ Number of rows    │ 20002  │ │ float64     │ 7     │                                    │
│ │ Number of columns │ 13     │ │ object      │ 5     │                                    │
│ └───────────────────┴────────┘ │ datetime64  │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃         ┃ missing ┃ complete ┃ mean  ┃ sd   ┃ p0    ┃ p25   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┃         ┃         ┃ rate     ┃       ┃      ┃       ┃       ┃       ┃       ┃        ┃  │
│ ┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ transac │       1 │        1 │ 10000 │ 5800 │     1 │  5000 │ 15000 │ 20000 │ ██████ │  │
│ │ tion_id │         │          │       │      │       │       │       │       │        │  │
│ │ product │       2 │        1 │    45 │   31 │     0 │    18 │    72 │   100 │ █▅▅▅▅▅ │  │
│ │ _id     │         │          │       │      │       │       │       │       │        │  │
│ │ custome │       2 │        1 │  1700 │ 1000 │     1 │   860 │  2600 │  5000 │ ████▁  │  │
│ │ r_id    │         │          │       │      │       │       │       │       │        │  │
│ │ online_ │     360 │     0.98 │   0.5 │  0.5 │     0 │     0 │     1 │     1 │ █    █ │  │
│ │ order   │         │          │       │      │       │       │       │       │        │  │
│ │ list_pr │       2 │        1 │  1100 │  580 │    12 │   580 │  1600 │  2100 │ ▄▆▄█▅▇ │  │
│ │ ice     │         │          │       │      │       │       │       │       │        │  │
│ │ standar │     200 │     0.99 │   560 │  410 │   7.2 │   220 │   800 │  1800 │ █▅▆▂▁▁ │  │
│ │ d_cost  │         │          │       │      │       │       │       │       │        │  │
│ │ product │     200 │     0.99 │ 38000 │ 2900 │ 33000 │ 36000 │ 41000 │ 43000 │ ▇▆▆▇▇█ │  │
│ │ _first_ │         │          │       │      │       │       │       │       │        │  │
│ │ sold_d  │         │          │       │      │       │       │       │       │        │  │
│ └─────────┴─────────┴──────────┴───────┴──────┴───────┴───────┴───────┴───────┴────────┘  │
│                                         datetime                                          │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓  │
│ ┃                   ┃ missing  ┃ complete rate  ┃ first       ┃ last       ┃ frequency ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩  │
│ │ transaction_date  │        2 │              1 │ 2017-01-01  │ 2017-12-30 │ None      │  │
│ └───────────────────┴──────────┴────────────────┴─────────────┴────────────┴───────────┘  │
╰─────────────────────────────────────────── End ───────────────────────────────────────────╯

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
93,94.0,86.0,1.0,2017-12-23,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,38482.0
3764,3765.0,38.0,1.0,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,39526.0
5156,5157.0,47.0,1.0,2017-05-11,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,37823.0
9784,9785.0,72.0,1.0,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,37873.0
13423,13424.0,2.0,1.0,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,38573.0
13643,13644.0,25.0,1.0,2017-05-19,0.0,Approved,Giant Bicycles,Road,medium,medium,1538.99,829.65,33552.0
14485,14486.0,23.0,1.0,2017-03-27,0.0,Approved,Norco Bicycles,Mountain,low,small,688.63,612.88,33455.0
14930,14931.0,31.0,1.0,2017-12-14,1.0,Approved,Giant Bicycles,Standard,medium,medium,230.91,173.18,37337.0
15662,15663.0,32.0,1.0,2017-06-04,1.0,Approved,Giant Bicycles,Standard,medium,medium,642.70,211.37,40618.0
16422,16423.0,9.0,1.0,2017-12-09,1.0,Approved,OHM Cycles,Road,medium,medium,742.54,667.40,37838.0


In [47]:
# Investigate if there multiple transactions recorded 
trans_cleaned.transaction_id.duplicated().value_counts()

False    20001
True         1
Name: transaction_id, dtype: int64

In [48]:
# Convert product_first_sold_date to the right format 
trans_cleaned['product_first_sold_date'] = pd.to_datetime(trans_cleaned['product_first_sold_date'], 
                                           unit = 'D',
                                           origin ='1899-12-30' )

trans_cleaned.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1.0,2.0,2950.0,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2.0,3.0,3120.0,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3.0,37.0,402.0,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4.0,88.0,3135.0,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5.0,78.0,787.0,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [49]:
#Convert online type to Boolean
def changeBoolean(var):
    if var == 0.0:
        return False
    elif var == 1.0:
        return True
    
trans_cleaned['online_order'] = trans_cleaned['online_order'].apply(changeBoolean)
trans_cleaned.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1.0,2.0,2950.0,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2.0,3.0,3120.0,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3.0,37.0,402.0,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4.0,88.0,3135.0,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5.0,78.0,787.0,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [50]:
# ADD a profit column and profit ratio
trans_cleaned['profit']      = trans_cleaned['list_price'] - trans_cleaned['standard_cost']
trans_cleaned['profitRatio'] = trans_cleaned['list_price'] / trans_cleaned['standard_cost']

# drop columns with null profit 
trans_cleaned          = trans_cleaned[trans_cleaned['profit'].notna()]
trans_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19803 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           19803 non-null  float64       
 1   product_id               19803 non-null  float64       
 2   customer_id              19803 non-null  float64       
 3   transaction_date         19803 non-null  datetime64[ns]
 4   online_order             19445 non-null  object        
 5   order_status             19803 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               19803 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [51]:
## ADD AN UNSPECIFIED COLUMN TO online_order that is na
trans_cleaned.online_order.fillna('Unspecified', inplace=True)

# Find all categorical columns 
columns = [i for i in list(trans_cleaned.columns) if trans_cleaned[i].dtype == 'O']

for item in columns:
    print(f'{item} unique Values {trans_cleaned[item].unique()}')


online_order unique Values [False True 'Unspecified']
order_status unique Values ['Approved' 'Cancelled']
brand unique Values ['Solex' 'Trek Bicycles' 'OHM Cycles' 'Norco Bicycles' 'Giant Bicycles'
 'WeareA2B']
product_line unique Values ['Standard' 'Road' 'Mountain' 'Touring']
product_class unique Values ['medium' 'low' 'high' ' ']
product_size unique Values ['medium' 'large' 'small']


In [52]:
# Observe thatcertain transactions were cancelled and note that Cancelled transactions dont add up to the revenue
trans_cleaned.order_status.value_counts() 

# Use only approved transcations 
trans_cleaned = trans_cleaned[trans_cleaned.order_status == 'Approved']

#### THE  RELEVANT METRICS I USE WILL BE 
- TOTAL AND AVERAGE REVENUE PER CUSTOMER 
- FREQUENCY OF TRANSCATION 
- AVERAGE PROFIT RATION  PER TRANSACTION 

In [53]:
transcMetrics = trans_cleaned.groupby('customer_id').agg({'profit' : ['sum', 'mean'],
                                          "customer_id" : 'count' ,
                                          'profitRatio' : ['sum', 'mean']})



#### DATA CLEANING
ALL DATA SETS ARE CLEANED INORDER TO BE PREPROCESSED.
REDUNDANT COLUMNS ARE DROPPED, MISSING VALUES ARE FILLED, CATEGORIES ARE CORRECTED. 

#### DATA WRANGLING
DATA IS WRANGLED TO FIT INTO ONE ONE DATASET 

#### COMPUTATION
- COMPUTE AGE AND CLASSIFY BY AGE BRACKET

- AN APPROPRIATE METHOD IS USED TO CLASSIFY CURRENT TARGET- 1,0 WHICH WILL CORRESEPOND TO TRUE OR FALSE
  PROBLEM?? -- EHAT METHOD CAN I USE TO DESIGN A MODEL THAT CAN EFFECTIVELY CATEGORISE TH

#### VISUALIZATION AND CLASSIFICATION OF PUCAHSE BEHAVIOURS 
EDA IS CARRIED OUT O THE EXISTING DATASET AND MORE UNDERSTANDING OF THE PURCHASE BEHAVIOURS IS ATTAINED 

#### FEATURE ENGINEERING 
THE DATA SET IS ACCUSTOMED TO BE FIT INTO A MODEL

#### DEVELOP MODEL 
IN THIS CASE, WE WILL USE A CLASSICAL CLASSIFICTION MODEL TO CLASSIFY CUSKTOMERS INTO TWO GROUPS BASED ON THEIR PURCHASE BEHAVIOUS 

#### MODEL TESTING AND VALIDATION 
THE MODEL IS TESTED WITH THE TEST DATA TO VALIDATE THE CORRECTIES NESSA ND ACCUERACY

#### MODEL DEPLOYMENT
APPLY MODEL ON THE NEW CUSTOMERS FOR CLASSIFICATION